In [ ]:
import os
import pandas as pd
import pickle
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset
from datetime import timedelta
from utils import highlight_print, save_memory

def prepare_datasets(save_dir='../dataset/'):
    os.makedirs(save_dir, exist_ok=True)

    levels = [
        [],                        # Level 1: Total
        ['state_id'],              # Level 2: State
        ['store_id'],              # Level 3: Store
        ['cat_id'],                # Level 4: Category
        ['dept_id'],               # Level 5: Department
        ['state_id', 'cat_id'],    # Level 6: State-Category
        ['state_id', 'dept_id'],   # Level 7: State-Department
        ['store_id', 'cat_id'],    # Level 8: Store-Category
        ['store_id', 'dept_id'],   # Level 9: Store-Department
        ['item_id'],               # Level 10: Item
        ['item_id', 'state_id'],   # Level 11: Item-State
        ['item_id', 'store_id']    # Level 12: Individual
    ]

    for level_idx, level in enumerate(levels, start=1):

        datasets = {'train': {}, 'test': {}}

        highlight_print(f"Preparing dataset for level {level_idx}")

        # 데이터 로드
        agg_df = pd.read_csv(f'../data/preprocessed/agg_df_level_{level_idx}.csv')
        calendar_df = pd.read_csv('../data/preprocessed/calendar_df.csv')

        # 메모리 절약
        agg_df = save_memory(agg_df)
        calendar_df = save_memory(calendar_df)

        # 날짜 처리
        start_date = pd.to_datetime('2011-01-29')
        agg_df['d'] = agg_df['d'].apply(lambda x: int(x.split('_')[1]) - 1)
        agg_df['d'] = start_date + pd.to_timedelta(agg_df['d'], unit='D')
        calendar_df['d'] = calendar_df['d'].apply(lambda x: int(x.split('_')[1]) - 1)
        calendar_df['d'] = start_date + pd.to_timedelta(calendar_df['d'], unit='D')
        
        # ID 컬럼 생성
        if len(level) == 0:
            agg_df.insert(1, 'id', 'total')
        elif len(level) == 1:
            agg_df.insert(1, 'id', agg_df[level[0]])
        elif len(level) > 1:
            agg_df.insert(1, 'id', agg_df[level[0]] + '_' + agg_df[level[1]])

        # ID로 그룹 인코딩
        groups = agg_df['id'].unique()
        group_encoder = {group: group_idx for group_idx, group in enumerate(groups)}
 
        # 그룹 데이터 생성
        group_df = agg_df.merge(calendar_df, on="d", how="left")

        # 데이터셋 생성
        train_dataset = ListDataset(
            [
                {
                    FieldName.ITEM_ID: id,
                    FieldName.TARGET: group["sales_sum"].values[:-28],
                    FieldName.START: pd.Period(min(group["d"]), freq="1D"),
                    FieldName.FEAT_STATIC_CAT: [group_encoder[id]],
                    FieldName.FEAT_DYNAMIC_REAL: group[[
                        "sales_mean", "sales_std", "sales_max", "sales_min", 
                        "sales_lag1", "sales_lag7", "sales_lag28",  
                        "sales_rolling7_mean", "sales_rolling28_mean", 
                        "sales_trend", 
                        "release_mean", "out_of_stock_mean", 
                        "sell_price_mean", "sell_price_std", "sell_price_max", "sell_price_min", 
                        "sell_price_diff", "sell_price_trend", "sell_price_in_store_mean",
                        "year_delta", "quarter_sin", "quarter_cos", "month_sin", "month_cos",  
                        "day_sin", "day_cos", "weekday_sin", "weekday_cos",
                        'event_count'
                    ]].values[:-28].T,
                    FieldName.FEAT_DYNAMIC_CAT: group[[
                        'snap_CA', 'snap_TX', 'snap_WI', 
                        'event_name_1_enc', 'event_name_2_enc', 
                        'event_type_1_enc', 'event_type_2_enc'
                    ]].values[:-28].T,
                }
                for id, group in group_df.groupby("id")
            ],
            freq="D",
        )
        test_dataset = ListDataset(
            [
                {
                    FieldName.ITEM_ID: id,
                    FieldName.TARGET: group["sales_sum"].values[:], 
                    FieldName.START: pd.Period(min(group["d"]), freq="1D"),
                    FieldName.FEAT_STATIC_CAT: [group_encoder[id]],
                    FieldName.FEAT_DYNAMIC_REAL: group[[
                        "sales_mean", "sales_std", "sales_max", "sales_min", 
                        "sales_lag1", "sales_lag7", "sales_lag28",  
                        "sales_rolling7_mean", "sales_rolling28_mean", 
                        "sales_trend", 
                        "release_mean", "out_of_stock_mean", 
                        "sell_price_mean", "sell_price_std", "sell_price_max", "sell_price_min", 
                        "sell_price_diff", "sell_price_trend", "sell_price_in_store_mean", 
                        "year_delta", "quarter_sin", "quarter_cos", "month_sin", "month_cos",  
                        "day_sin", "day_cos", "weekday_sin", "weekday_cos",
                        'event_count'
                    ]].values[:].T,  
                    FieldName.FEAT_DYNAMIC_CAT: group[[
                        'snap_CA', 'snap_TX', 'snap_WI', 
                        'event_name_1_enc', 'event_name_2_enc', 
                        'event_type_1_enc', 'event_type_2_enc'
                    ]].values[:].T,  
                }
                for id, group in group_df.groupby("id")
            ],
            freq="D",
        )   
        
        datasets['train'] = train_dataset
        datasets['test'] = test_dataset
    
        with open(os.path.join(save_dir, f'dataset_level_{level_idx}.pkl'), 'wb') as f:
            pickle.dump(datasets, f)

prepare_datasets()

In [12]:
def prepare_datasets(save_dir='../dataset/'):
    os.makedirs(save_dir, exist_ok=True)

    levels = [
        [],                        # Level 1: Total
        ['state_id'],              # Level 2: State
        ['store_id'],              # Level 3: Store
        ['cat_id'],                # Level 4: Category
        ['dept_id'],               # Level 5: Department
        ['state_id', 'cat_id'],    # Level 6: State-Category
        ['state_id', 'dept_id'],   # Level 7: State-Department
        ['store_id', 'cat_id'],    # Level 8: Store-Category
        ['store_id', 'dept_id'],   # Level 9: Store-Department
        ['item_id'],               # Level 10: Item
        ['item_id', 'state_id'],   # Level 11: Item-State
        ['item_id', 'store_id']    # Level 12: Individual
    ]

    for level_idx, level in enumerate(levels, start=1):
        datasets = {'train': {}, 'test': {}}
        
        highlight_print(f"Preparing dataset for level {level_idx}")

        # 데이터 로드
        agg_df = pd.read_csv(f'../data/preprocessed/agg_df_level_{level_idx}.csv')
        calendar_df = pd.read_csv('../data/preprocessed/calendar_df.csv')

        # 메모리 절약
        agg_df = save_memory(agg_df)
        calendar_df = save_memory(calendar_df)

        # 날짜 처리
        start_date = pd.to_datetime('2011-01-29')
        agg_df['d'] = agg_df['d'].apply(lambda x: int(x.split('_')[1]) - 1)
        agg_df['d'] = start_date + pd.to_timedelta(agg_df['d'], unit='D')
        calendar_df['d'] = calendar_df['d'].apply(lambda x: int(x.split('_')[1]) - 1)
        calendar_df['d'] = start_date + pd.to_timedelta(calendar_df['d'], unit='D')
        
        # ID 컬럼 생성
        if len(level) == 0:
            agg_df.insert(1, 'id', 'total')
        elif len(level) == 1:
            agg_df.insert(1, 'id', agg_df[level[0]])
        elif len(level) > 1:
            agg_df.insert(1, 'id', agg_df[level[0]] + '_' + agg_df[level[1]])

        # ID로 그룹 인코딩
        groups = agg_df['id'].unique()
        group_encoder = {group: group_idx for group_idx, group in enumerate(groups)}
 
        # 그룹 데이터 생성
        group_df = agg_df.merge(calendar_df, on="d", how="left")

        # 데이터셋 생성
        train_dataset = ListDataset(
            [
                {
                    FieldName.ITEM_ID: id,
                    FieldName.TARGET: group["sales_sum"].values[:-28],
                    FieldName.START: pd.Period(min(group["d"]), freq="1D"),
                    'id': [group_encoder[id]],
                    "sales_mean": group["sales_mean"].values[:-28].reshape(1, -1),
                    "sales_std": group["sales_std"].values[:-28].reshape(1, -1),
                    "sales_max": group["sales_max"].values[:-28].reshape(1, -1),
                    "sales_min": group["sales_min"].values[:-28].reshape(1, -1),
                    "sales_lag1": group["sales_lag1"].values[:-28].reshape(1, -1),
                    "sales_lag7": group["sales_lag7"].values[:-28].reshape(1, -1),
                    "sales_lag28": group["sales_lag28"].values[:-28].reshape(1, -1),
                    "sales_rolling7_mean": group["sales_rolling7_mean"].values[:-28].reshape(1, -1),
                    "sales_rolling28_mean": group["sales_rolling28_mean"].values[:-28].reshape(1, -1),
                    "sales_trend": group["sales_trend"].values[:-28].reshape(1, -1),
                    "release_mean": group["release_mean"].values[:-28].reshape(1, -1),
                    "out_of_stock_mean": group["out_of_stock_mean"].values[:-28].reshape(1, -1),
                    "sell_price_mean": group["sell_price_mean"].values[:-28].reshape(1, -1),
                    "sell_price_std": group["sell_price_std"].values[:-28].reshape(1, -1),
                    "sell_price_max": group["sell_price_max"].values[:-28].reshape(1, -1),
                    "sell_price_min": group["sell_price_min"].values[:-28].reshape(1, -1),
                    "sell_price_diff": group["sell_price_diff"].values[:-28].reshape(1, -1),
                    "sell_price_trend": group["sell_price_trend"].values[:-28].reshape(1, -1),
                    "sell_price_in_store_mean": group["sell_price_in_store_mean"].values[:-28].reshape(1, -1),
                    "year_delta": group["year_delta"].values[:-28].reshape(1, -1),
                    "quarter_sin": group["quarter_sin"].values[:-28].reshape(1, -1),
                    "quarter_cos": group["quarter_cos"].values[:-28].reshape(1, -1),
                    "month_sin": group["month_sin"].values[:-28].reshape(1, -1),
                    "month_cos": group["month_cos"].values[:-28].reshape(1, -1),
                    "day_sin": group["day_sin"].values[:-28].reshape(1, -1),
                    "day_cos": group["day_cos"].values[:-28].reshape(1, -1),
                    "weekday_sin": group["weekday_sin"].values[:-28].reshape(1, -1),
                    "weekday_cos": group["weekday_cos"].values[:-28].reshape(1, -1),
                    "event_count": group["event_count"].values[:-28].reshape(1, -1),
                    'snap_CA': group['snap_CA'].values[:-28],
                    'snap_TX': group['snap_TX'].values[:-28],
                    'snap_WI': group['snap_WI'].values[:-28],
                    'event_name_1_enc': group['event_name_1_enc'].values[:-28],
                    'event_name_2_enc': group['event_name_2_enc'].values[:-28],
                    'event_type_1_enc': group['event_type_1_enc'].values[:-28],
                    'event_type_2_enc': group['event_type_2_enc'].values[:-28]
                }
                for id, group in group_df.groupby("id")
            ],
            freq="D",
        )

        test_dataset = ListDataset(
            [
                {
                    FieldName.ITEM_ID: id,
                    FieldName.TARGET: group["sales_sum"].values[:], 
                    FieldName.START: pd.Period(min(group["d"]), freq="1D"),
                    'id': [group_encoder[id]],
                    "sales_mean": group["sales_mean"].values[:].reshape(1, -1),
                    "sales_std": group["sales_std"].values[:].reshape(1, -1),
                    "sales_max": group["sales_max"].values[:].reshape(1, -1),
                    "sales_min": group["sales_min"].values[:].reshape(1, -1),
                    "sales_lag1": group["sales_lag1"].values[:].reshape(1, -1),
                    "sales_lag7": group["sales_lag7"].values[:].reshape(1, -1),
                    "sales_lag28": group["sales_lag28"].values[:].reshape(1, -1),
                    "sales_rolling7_mean": group["sales_rolling7_mean"].values[:].reshape(1, -1),
                    "sales_rolling28_mean": group["sales_rolling28_mean"].values[:].reshape(1, -1),
                    "sales_trend": group["sales_trend"].values[:].reshape(1, -1),
                    "release_mean": group["release_mean"].values[:].reshape(1, -1),
                    "out_of_stock_mean": group["out_of_stock_mean"].values[:].reshape(1, -1),
                    "sell_price_mean": group["sell_price_mean"].values[:].reshape(1, -1),
                    "sell_price_std": group["sell_price_std"].values[:].reshape(1, -1),
                    "sell_price_max": group["sell_price_max"].values[:].reshape(1, -1),
                    "sell_price_min": group["sell_price_min"].values[:].reshape(1, -1),
                    "sell_price_diff": group["sell_price_diff"].values[:].reshape(1, -1),
                    "sell_price_trend": group["sell_price_trend"].values[:].reshape(1, -1),
                    "sell_price_in_store_mean": group["sell_price_in_store_mean"].values[:].reshape(1, -1),
                    "year_delta": group["year_delta"].values[:].reshape(1, -1),
                    "quarter_sin": group["quarter_sin"].values[:].reshape(1, -1),
                    "quarter_cos": group["quarter_cos"].values[:].reshape(1, -1),
                    "month_sin": group["month_sin"].values[:].reshape(1, -1),
                    "month_cos": group["month_cos"].values[:].reshape(1, -1),
                    "day_sin": group["day_sin"].values[:].reshape(1, -1),
                    "day_cos": group["day_cos"].values[:].reshape(1, -1),
                    "weekday_sin": group["weekday_sin"].values[:].reshape(1, -1),
                    "weekday_cos": group["weekday_cos"].values[:].reshape(1, -1),
                    "event_count": group["event_count"].values[:].reshape(1, -1),
                    'snap_CA': group['snap_CA'].values[:],
                    'snap_TX': group['snap_TX'].values[:],
                    'snap_WI': group['snap_WI'].values[:],
                    'event_name_1_enc': group['event_name_1_enc'].values[:],
                    'event_name_2_enc': group['event_name_2_enc'].values[:],
                    'event_type_1_enc': group['event_type_1_enc'].values[:],
                    'event_type_2_enc': group['event_type_2_enc'].values[:]
                }
                for id, group in group_df.groupby("id")
            ],
            freq="D",
        )   
        
        datasets['train'] = train_dataset
        datasets['test'] = test_dataset
    
        with open(os.path.join(save_dir, f'dataset_level_{level_idx}.pkl'), 'wb') as f:
            pickle.dump(datasets, f)

prepare_datasets()

Preparing dataset for level 1
Preparing dataset for level 2
Preparing dataset for level 3
Preparing dataset for level 4
Preparing dataset for level 5
Preparing dataset for level 6
Preparing dataset for level 7
Preparing dataset for level 8
Preparing dataset for level 9
Preparing dataset for level 10
Preparing dataset for level 11
Preparing dataset for level 12
